# Функции и работа с данными

Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
	- оценка 2 и меньше - низкий рейтинг
	- оценка 4 и меньше - средний рейтинг
	- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
ratings_new = ratings.groupby('movieId').mean()['rating'].reset_index()
ratings_new.head()

,movieId,rating
0,1,3.920930
1,2,3.431818
2,3,3.259615
3,4,2.357143
4,5,3.071429


In [5]:
ratings_with_movies = ratings_new.merge(movies, how = 'left', on = 'movieId')
ratings_with_movies.head()

,movieId,rating,title,genres
0,1,3.920930,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,3.431818,Jumanji (1995),Adventure|Children|Fantasy
2,3,3.259615,Grumpier Old Men (1995),Comedy|Romance
3,4,2.357143,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,3.071429,Father of the Bride Part II (1995),Comedy


In [6]:
ratings_new.shape

(9724, 2)

In [7]:
ratings_with_movies.shape

(9724, 4)

In [8]:
def raiting_class(row):
    if row.rating <= 2:
        return 'низкий рейтинг'
    elif row.rating <= 4:
        return 'средний рейтинг'
    elif (row.rating == 4.5) or (row.rating == 5):
        return 'высокий рейтинг'
    else:
        return 'undefined'

In [9]:
ratings_with_movies['class'] = ratings_with_movies.apply(raiting_class, axis=1)

In [10]:
ratings_with_movies.head()

,movieId,rating,title,genres,class
0,1,3.920930,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,средний рейтинг
1,2,3.431818,Jumanji (1995),Adventure|Children|Fantasy,средний рейтинг
2,3,3.259615,Grumpier Old Men (1995),Comedy|Romance,средний рейтинг
3,4,2.357143,Waiting to Exhale (1995),Comedy|Drama|Romance,средний рейтинг
4,5,3.071429,Father of the Bride Part II (1995),Comedy,средний рейтинг


In [11]:
ratings_with_movies['class'].value_counts()

средний рейтинг    7385
низкий рейтинг     1079
высокий рейтинг     671
undefined           589
Name: class, dtype: int64

Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.

In [12]:
keywords = pd.read_csv('keywords.csv')
keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [13]:
geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

In [14]:
def geo_classification(row):
    for key, value in geo_data.items():
        for i in value:
            if i in row.keyword:
                return key
    return 'undefined'

In [15]:
keywords['region'] = keywords.apply(geo_classification, axis=1)

In [16]:
keywords['region'].value_counts()

undefined         99253
Центр               387
Северо-Запад        266
Дальний Восток       94
Name: region, dtype: int64

In [17]:
keywords[keywords['region'] == 'Дальний Восток'].tail()

,keyword,shows,region
90402,аэропорт владивосток,4658,Дальний Восток
92442,монастырев владивосток официальный сайт,3870,Дальний Восток
92843,проститутки хабаровск,4120,Дальний Восток
93872,биглион хабаровск,3890,Дальний Восток
95572,точное время владивосток,4142,Дальний Восток


Задание 3 (бонусное)
Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
	- для каждой строки пройдите по всем годам списка years
	- если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
	- если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [18]:
years = list(range(1950, 2011))
print(years)

[1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010]


In [19]:
ratings_with_movies_new = ratings.merge(movies, how = 'left', on = 'movieId')
ratings_with_movies_new.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [20]:
def production_year(row):
    for i in years:
        if str(i) in row.title:
            return str(i)
    return '1900'

In [21]:
ratings_with_movies_new['year'] = ratings_with_movies_new.apply(production_year, axis=1)

In [22]:
ratings_with_movies_new.head()

,userId,movieId,rating,timestamp,title,genres,year
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,1995
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller,1995
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1995
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1995


In [23]:
ratings_with_movies_new.groupby('year').mean()['rating'].sort_values(ascending=False).reset_index().head()

,year,rating
0,1957,4.039535
1,1954,4.009191
2,1962,3.969466
3,1952,3.953125
4,1972,3.944293


In [24]:
ratings_with_movies_new[ratings_with_movies_new['year'] == '1957'].groupby('title').mean()['rating']. \
sort_values(ascending=False).reset_index().head()

,title,rating
0,20 Million Miles to Earth (1957),5.000000
1,My Man Godfrey (1957),5.000000
2,Paths of Glory (1957),4.541667
3,Witness for the Prosecution (1957),4.388889
4,"Nights of Cabiria (Notti di Cabiria, Le) (1957)",4.285714


In [26]:
ratings_with_movies_new[ratings_with_movies_new['title'] == 'My Man Godfrey (1957)']

,userId,movieId,rating,timestamp,title,genres,year
22982,156,3096,5.0,946799236,My Man Godfrey (1957),Comedy,1957
